# Brazil Temperature Data 

### Columns present across different data sources : 
- **DateValueCET**- sets the date for specific measurement
- **TimeValueCET** - sets the hour for the specific measurement

Relavant Columns:
- **Value:** which is the actual measurement
- **Unit:** values(Celsius)
- **PowerPriceAreaCode:** values (BRAZIL_SOUTH, BRAZIL_NORTH, BRAZIL_NORTHEAST, RAZIL_SOUTHEAST_CENTRALWEST)

In [ ]:
import pandas as pd 
from get_files_only import get_file_from_xdrive
# Read the Excel file
df_temp = get_file_from_xdrive("Temperature_Data.xlsx")
df_temp = df_temp.rename(columns = {"Value": "Temperature"})
df_temp = df_temp[['DateValueCET', 'TimeValueCET', 'PowerPriceAreaCode', 'Temperature','Unit']]
df_temp['DateTime'] = pd.to_datetime((df_temp['DateValueCET']).astype(str) + ' ' + (df_temp['TimeValueCET']).astype(str), format='%Y-%m-%d %H:%M:%S')
df_temp['DateTime'] = pd.to_datetime(df_temp['DateTime'], format='%Y-%m-%d %H:%M:%S')
df_temp['Month'] = df_temp['DateTime'].dt.month
df_temp = df_temp.groupby(['DateTime', 'PowerPriceAreaCode']).mean().reset_index()

display(df_temp.groupby(['Month','PowerPriceAreaCode']).describe())
#df_temp.drop(columns={'Month'})

C:\Users\veron\AppData\Local\Temp\ipykernel_30592\2399143368.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_temp = df_temp.groupby(['DateTime', 'PowerPriceAreaCode']).mean().reset_index()


Temperature                       \
                                         count       mean       std   
Month PowerPriceAreaCode                                              
8.0   BRAZIL_NORTH                       146.0  27.973823  2.803696   
      BRAZIL_NORTHEAST                   146.0  24.474877  1.283118   
      BRAZIL_SOUTH                       146.0  13.225595  4.229925   
      BRAZIL_SOUTHEAST_CENTRALWEST       146.0  17.781668  3.742463   
9.0   BRAZIL_NORTH                       651.0  28.516166  2.916569   
      BRAZIL_NORTHEAST                   651.0  24.897494  1.390063   
      BRAZIL_SOUTH                       651.0  18.204726  3.601527   
      BRAZIL_SOUTHEAST_CENTRALWEST       651.0  22.804747  3.442217   

                                                                     \
                                          min        25%        50%   
Month PowerPriceAreaCode                                              
8.0   BRAZIL_NORTH                  24.570714  25.319583  27.221667   
      BRAZIL_NORTHEAST              22.844524  23.241607  24.247857   
      BRAZIL_SOUTH                   5.801667   9.871667  13.290119   
      BRAZIL_SOUTHEAST_CENTRALWEST  12.224524  14.724924  17.001573   
9.0   BRAZIL_NORTH                  24.538810  25.767500  27.728750   
      BRAZIL_NORTHEAST              22.700952  23.590000  24.646190   
      BRAZIL_SOUTH                  10.446905  15.511823  17.907381   
      BRAZIL_SOUTHEAST_CENTRALWEST  16.254524  19.903333  22.133488   

                                                          
                                          75%        max  
Month PowerPriceAreaCode                                  
8.0   BRAZIL_NORTH                  30.770893  33.031628  
      BRAZIL_NORTHEAST              25.696250  27.065952  
      BRAZIL_SOUTH                  15.781192  22.026279  
      BRAZIL_SOUTHEAST_CENTRALWEST  20.222321  26.160465  
9.0   BRAZIL_NORTH                  31.306310  33.865385  
      BRAZIL_NORTHEAST              26.273500  27.373077  
      BRAZIL_SOUTH                  20.393305  29.320930  
      BRAZIL_SOUTHEAST_CENTRALWEST  25.656374  30.506744

In [19]:
import plotly.express as px

fig_1 = px.line( 
    df_temp,
    x='DateTime',
    y='Temperature',
    color='PowerPriceAreaCode',
    title=f'Temperature in Celsius Over Time by Region for 2024',
    labels={'Temperature in Celsius': 'Temperature in Celsius', 'DateTime': 'Date and Time'}
)

""" # Update layout for better visibility
fig.update_layout(
    xaxis_title='Date and Time',
    yaxis_title='Rainfall Potential (GWh)',
    legend_title='Region',
    xaxis=dict(tickformat="%Y-%m-%d %H:%M")
) """

fig_1.show()

## Daily Average Temperature

In [13]:

df_temp = df_temp.drop(columns=['Month'])
# Convert 'DateTime' to date only (without time) for daily grouping
df_temp['Date'] = df_temp['DateTime'].dt.date

# Group by 'Date' and 'PowerPriceAreaCode' to calculate the daily mean
df_daily_mean = df_temp.groupby(['Date', 'PowerPriceAreaCode']).mean()

# Merge the daily mean back to the original DataFrame on 'Date' and 'PowerPriceAreaCode'
daily_merged = pd.merge(df_temp, df_daily_mean, on=['Date', 'PowerPriceAreaCode'])

daily_merged= daily_merged.rename(columns = {'Temperature_x': 'Hourly_Temperature', 'Temperature_y': 'Daily_Average_Temperature'})

#all_values =  pd.merge(merged_df, daily_merged, on=['PowerPriceAreaCode', 'DateTime'], how='outer')
#data_2023_SE_with_temperature = all_values[(all_values['PowerPriceAreaCode'] == 'BRAZIL_SOUTHEAST_CENTRALWEST') & (all_values['DateTime'].dt.year == 2024)]


display(daily_merged.head())

C:\Users\veron\AppData\Local\Temp\ipykernel_30592\1721491325.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_daily_mean = df_temp.groupby(['Date', 'PowerPriceAreaCode']).mean()


,DateTime,PowerPriceAreaCode,Hourly_Temperature,Date,Daily_Average_Temperature
0,2024-08-25 22:00:00,BRAZIL_NORTH,30.867391,2024-08-25,30.273815
1,2024-08-25 23:00:00,BRAZIL_NORTH,29.680238,2024-08-25,30.273815
2,2024-08-25 22:00:00,BRAZIL_NORTHEAST,25.994286,2024-08-25,25.656310
3,2024-08-25 23:00:00,BRAZIL_NORTHEAST,25.318333,2024-08-25,25.656310
4,2024-08-25 22:00:00,BRAZIL_SOUTH,11.598148,2024-08-25,11.176931


In [20]:
import plotly.express as px

fig_1 = px.line( 
    daily_merged,
    x='DateTime',
    y='Daily_Average_Temperature',
    color='PowerPriceAreaCode',
    title=f'Daily Average Temperature in Celsius Over Time by Region for 2024',
    labels={'Daily Average Temperature in Celsius': 'Daily Average Temperature in Celsius', 'DateTime': 'Date and Time'}
)

""" # Update layout for better visibility
fig.update_layout(
    xaxis_title='Date and Time',
    yaxis_title='Rainfall Potential (GWh)',
    legend_title='Region',
    xaxis=dict(tickformat="%Y-%m-%d %H:%M")
) """

fig_1.show()